<a href="https://colab.research.google.com/github/mmdedavoodi/mnist-pytorch-classifier/blob/main/Minist_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
transform_train = transforms.Compose([transforms.ToTensor(),
                                      transforms.Normalize(0.13 , 0.31),
                                      transforms.RandomRotation(30),
                                      transforms.RandomAffine(10 , translate= (0.1 , 0.1))
                                     ])
transform_test = transforms.Compose([transforms.ToTensor(),
                                      transforms.Normalize(0.13 , 0.31)])


dataset_train = torchvision.datasets.MNIST(root="data",
                                           train = True,
                                           download = True,
                                           transform = transform_train)

dataset_test = torchvision.datasets.MNIST(root="data",
                                           train = False,
                                           download = True,
                                           transform = transform_test)



In [ ]:
dataset_train

##Test DataSet

In [ ]:
idx = 8
print(dataset_train[idx][0].size())
plt.imshow(dataset_train[idx][0].permute(1,2,0) , cmap="gray")
print(dataset_train[idx][1])

In [ ]:
28 * 28

##Data Loader

In [ ]:
dataloader_train = torch.utils.data.DataLoader(dataset_train , batch_size=64 , shuffle = True , num_workers=2)
dataloader_test = torch.utils.data.DataLoader(dataset_test , batch_size=64 , shuffle = False , num_workers=2)

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(784 , 512),
    torch.nn.ReLU(),
    torch.nn.Linear(512 , 128),
    torch.nn.ReLU(),
    torch.nn.Linear(128 , 10),
)

In [ ]:
model

In [ ]:
optimizer = torch.optim.Adam(model.parameters())
loss_function = torch.nn.CrossEntropyLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
from tqdm import tqdm_notebook as tqdm

In [ ]:
model(dataset_train[idx][0])

In [ ]:
model.to(device)

In [ ]:
train_loss = []
train_acc = []
test_acc = []
test_loss = []


for epoch in range(30):
  model.train()
  pbar = tqdm(dataloader_train , desc = f"Training epoch {epoch + 1}")

  sum_correct = 0
  sum_sample = 0
  sum_loss = 0
  for x , y in pbar:
    x = x.to(device)
    y = y.to(device)
    optimizer.zero_grad()

    output = model(x)
    loss_value = loss_function(output , y)
    pbar.set_postfix_str(f"Loss = {loss_value:.3f}")
    loss_value.backward()
    optimizer.step()


    sum_correct += (output.argmax(axis = 1) == y).sum().item()
    sum_sample += len(y)
    sum_loss += loss_value * len(y)

  train_loss.append(sum_loss / sum_sample)
  train_acc.append(sum_correct / sum_sample)
  print(f"Average train loss: {sum_loss / sum_sample:.4f}")
  print(f"Average train Accuracy: {sum_correct / sum_sample:.4f}")
  print("--------------------------------")

  pbar = tqdm(dataloader_test , desc  = f"Testing in epoch {epoch + 1}" )

  for x , y in pbar:
    model.eval()
    x = x.to(device)
    y = y.to(device)

    with torch.no_grad():
        output = model(x)
        loss_value = loss_function(output , y)
        pbar.set_postfix_str(f"Loss = {loss_value:.3f}")
        sum_sample += len(y)
        sum_loss += loss_value.item() * len(y)
        sum_correct += (y == output.argmax(axis = 1)).sum().item()

  test_loss.append((sum_loss / sum_sample))
  test_acc.append((sum_correct/sum_sample))
  print(f"Average test loss: {sum_loss / sum_sample :.4f}")
  print(f"Average test Accuracy: {sum_correct / sum_sample:.4f}")
  print("--------------------------------")



In [ ]:
train_loss

In [ ]:
test_loss = list(map(lambda x:x.item() , test_loss))
train_loss = list(map(lambda x:x.item() , train_loss))

In [ ]:
plt.figure(figsize=(15,4))
plt.subplot(1,2,1)
plt.plot(train_acc , label = "Train Acc")
plt.plot(test_acc , label = "Test Acc")
plt.grid()
plt.legend()
plt.xlabel("Accuracy")
plt.ylabel("Epoch")

plt.subplot(1,2,2)
plt.plot(train_loss , label = "Train Loss")
plt.plot(test_loss , label = "Test Loss")
plt.grid()
plt.legend()
plt.xlabel("Loss")
plt.ylabel("Epoch")



In [ ]:
plt.imshow(dataset_test[0][0].permute(1,2,0))
print(dataset_test[0][1])


In [ ]:
uncorrect = []
correct = []
for x , y in dataset_test:
  x = x.to(device)

  output = model(x)
  output = output.argmax()

  if y != output:
    uncorrect.append((x , y , output.item()))
  elif y == output:
    correct.append((x , y , output.item()))



In [ ]:
def show_uncorrect(idx:int):

  print(f"Real label = {uncorrect[idx][1]}")
  print(f"Predicted label = {uncorrect[idx][2]}")
  plt.imshow(uncorrect[idx][0].to('cpu').permute(1,2,0) , cmap="gray")

def show_correct(idx:int):

  print(f"Real label = {correct[idx][1]}")
  print(f"Predicted label = {correct[idx][2]}")
  plt.imshow(correct[idx][0].to('cpu').permute(1,2,0) , cmap="gray")



In [ ]:
show_uncorrect(9)

In [ ]:
show_correct(9)